## Import and load packages

In [69]:
import pickle
import os
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib
import geopandas as gpd

# Set the directory
path = "/Users/Yannick/Documents/LUC_github/VOC_land_use"
os.chdir(path)

# Load the similarity_threshold_comparison_list from the file
with open('similarity_threshold_comparison_list.pkl', 'rb') as f:
    similarity_threshold_comparison_list = pickle.load(f)


def get_df_by_similarity(similarity_threshold, similarity_threshold_comparison_list):
    # Filter the list for the matching similarity_threshold
    for item in similarity_threshold_comparison_list:
        if item['similarity_threshold'] == similarity_threshold:
            return item['df']  # Return the DataFrame
    return None  # Return None if no match is found

combined_df = pd.DataFrame()
similarity_thresholds = set(range(0, 101))

for similarity_threshold in similarity_thresholds:
    df_for_threshold = get_df_by_similarity(similarity_threshold, similarity_threshold_comparison_list)
    df_for_threshold["similarity_threshold"] = similarity_threshold
    combined_df = pd.concat([combined_df, df_for_threshold])

# Replace spaces with underscores in the 'Region2' column
combined_df['Region2'] = combined_df['Region2'].str.replace(' ', '_')

print(combined_df.columns)
print(combined_df["Region2"].unique())
print(combined_df["Region1"].unique())

print("done")


Index(['placename', 'region', 'geometry', 'Village', 'similarity_score', 'ID',
       'label', 'Year', 'Region1', 'Region2', 'quantity', 'unit', 'sheet_name',
       'similarity_threshold'],
      dtype='object')
['happitigam_corle' 'hina_corle' 'alloetcoer_corle']
['not_specified' 'Jattiga pattoe' 'gara pattoe' 'oedeka pattoe'
 'gangebadde pattoe' 'mende pattoe' 'adigaar pattoe' 'adicani pattoe'
 'oedoegaha pattoe' 'dalia pattoe' 'dochegaha pattoe' 'ragam pattoe'
 'nigombo district']
done


In [89]:
app = dash.Dash(__name__)
server = app.server

# Define the fixed center and zoom level for the map
FIXED_CENTER_LAT = 7.1  # Set to the desired latitude
FIXED_CENTER_LON = 80.0  # Set to the desired longitude
FIXED_ZOOM = 10.5  # Set to the desired zoom level

# Define custom color map for the values in Region1
color_map_points = {
    'not_specified': '#ffffff',  # White
    'Jattiga pattoe': '#ff7f00',  # Orange
    'gara pattoe': '#d9d9d9',     # Gray
    'oedeka pattoe': '#d714c7',   # Purple
    'gangebadde pattoe': '#f0f248',# Yellow
    'mende pattoe': '#774747',    # Brown
    'adigaar pattoe': '#a6cee3',   # Light Blue
    'adicani pattoe': '#7f7f7f',   # Gray
    'oedoegaha pattoe': '#bcbd22', # Olive
    'dalia pattoe': '#d7191c',     # Red
    'dochegaha pattoe': '#0000ff', # Light Blue
    'ragam pattoe': '#ffbb78',     # Green
    'nigombo district': '#c5b0d5',  # Light Purple
}
color_map_polygons = {
    'not_specified': '#ffffff',  # White
    'Jattiga pattoe': '#ff7f00',  # Orange
    'gara pattoe': '#d9d9d9',     # Gray
    'oedeka pattoe': '#d714c7',   # Purple
    'gangebadde pattoe': '#f0f248',# Yellow
    'mende pattoe': '#774747',    # Brown
    'adigaar pattoe': '#a6cee3',   # Light Blue
    'adicani pattoe': '#7f7f7f',   # Gray
    'oedoegaha pattoe': '#bcbd22', # Olive
    'dalia pattoe': '#d7191c',     # Red
    'dochegaha pattoe': '#0000ff', # Light Blue
    'ragam pattoe': '#ffbb78',     # Green
    'nigombo district': '#c5b0d5',  # Light Purple
}
# Define the layout of the app
app.layout = html.Div([
    html.H1("Map of Historical LU in Ceylon", style={'text-align': 'center', 'font-family': 'Helvetica'}),
    html.Div([
        dcc.RadioItems(
            options=[
                {'label': 'All', 'value': 'all_regions'},
                {'label': 'Alloetcoer Corle', 'value': 'alloetcoer_corle'},
                {'label': 'Happitigam Corle', 'value': 'happitigam_corle'},
                {'label': 'Hina Corle', 'value': 'hina_corle'},
            ],
            value='all_regions',
            id='data-source-radio',
            style={'margin-right': '20px'}
        ),
        dcc.RadioItems(
            options=[
                {'label': 'Show Voronoi Polygons', 'value': 'yes'},
                {'label': 'Hide Voronoi Polygons', 'value': 'no'},
            ],
            value='yes',
            id='show_voronoi',
            style={'margin-right': '20px'}),
        dcc.Download(id="download-data"),
        html.Button("Download Data", id="download-button", n_clicks=0, style={'position': 'absolute', 'top': '50px', 'right': '50px'})
    ], style={'display': 'flex', 'font-family': 'Helvetica'}),

    dcc.Graph(id='sankey-diagram', style={'height': '80vh', 'width': '100%'}),

    
    dcc.Slider(
        id='threshold-slider',
        min=0,
        max=100,
        step=1,
        value=75,  # Set initial value to 75
        marks={i: f'{i}' for i in range(0, 101, 10)},  # Optional slider marks
        tooltip={"placement": "bottom", "always_visible": True}
    )
])

# Define callback to update the Sankey diagram based on the selected source and threshold
@app.callback(
    [Output('sankey-diagram', 'figure'),
     Output('download-data', 'data')],
    [Input('data-source-radio', 'value'),
     Input('download-button', 'n_clicks'),
     Input('threshold-slider', 'value'),
     Input('show_voronoi', 'value')]
)
def update_map(selected_source_type, n_clicks, selected_threshold, show_voronoi):
    
    # Filter the dataframe based on the selected threshold
    filtered_df = combined_df[combined_df['similarity_threshold'] == selected_threshold]

    # Define the path to the GeoPackage and layer name
    gpk_path_template = "maps/theissen/{}_theissen.gpkg"  # Template for the file path

    # Set the region based on the selected source type
    region_mapping = {
        'alloetcoer_corle': 'alloetcoer',
        'happitigam_corle': 'happitigam',
        'hina_corle': 'hina',
    }

    # Initialize an empty GeoDataFrame for polygons
    polygons_gdf = gpd.GeoDataFrame()

    # Check if the selected source type is in the mapping
    if selected_source_type in region_mapping:
        filtered_df = filtered_df[filtered_df["Region2"] == selected_source_type]
        # Load the corresponding GeoPackage for the selected region
        gpk_path = gpk_path_template.format(region_mapping[selected_source_type])
        try:
            polygons_gdf = gpd.read_file(gpk_path)
        except Exception as e:
            print(f"Error loading {gpk_path}: {e}")
    else:
        # Load all region polygons
        polygons_gdfs = []
        for region in region_mapping.keys():
            gpk_path = gpk_path_template.format(region_mapping[region])
            try:
                region_gdf = gpd.read_file(gpk_path)
                polygons_gdfs.append(region_gdf)
            except Exception as e:
                print(f"Error loading {gpk_path}: {e}")
        polygons_gdf = pd.concat(polygons_gdfs, ignore_index=True)

    # Convert the filtered dataframe into GeoDataFrame
    filtered_gdf = gpd.GeoDataFrame(
        filtered_df,
        geometry=filtered_df['geometry'],
        crs="EPSG:5234"  # The original CRS of your data
    )

    # Reproject to WGS84 (EPSG:4326) for plotting
    filtered_gdf = filtered_gdf.to_crs(epsg=4326)
    polygons_gdf = polygons_gdf.to_crs(epsg=4326)

    # Create scatter mapbox with fixed center and zoom level
    fig = px.scatter_mapbox(
        filtered_gdf,
        lat=filtered_gdf.geometry.y,
        lon=filtered_gdf.geometry.x,
        zoom=FIXED_ZOOM,
        center={"lat": FIXED_CENTER_LAT, "lon": FIXED_CENTER_LON},
        mapbox_style="carto-positron",  # Change to "carto-positron" if needed
        hover_name='label',  # Show 'label' column on hover
        color='Region1',  # Color points according to the 'Region1' column
        color_discrete_map=color_map_points  # Apply custom colors
    )
    
    if show_voronoi == 'yes':  # Ensure polygons_gdf is not empty
        for _, row in polygons_gdf.iterrows():
            region_color = color_map_polygons.get(row['Region1'], '#FFFFFF')  # Default to white if not found
            
            # Set transparency (0.3 for 30% opacity, adjust as needed)
            transparent_fillcolor = f'rgba{tuple(int(region_color[i:i+2], 16) for i in (1, 3, 5)) + (0.3,)}'  # Convert hex to RGBA
            
            if row.geometry.geom_type == 'Polygon':
                # If the geometry is a Polygon
                fig.add_trace(go.Scattermapbox(
                    lat=list(row.geometry.exterior.coords.xy[1]),  # Convert to list
                    lon=list(row.geometry.exterior.coords.xy[0]),  # Convert to list
                    mode='lines',  # Draw the polygon outline
                    fill='toself',  # Fill the polygon
                    fillcolor=transparent_fillcolor,  # Transparent fill color
                    line=dict(color=region_color, width=1),  # Outline color and width
                    name=row['Region1']  # Optional: Name for the legend
                ))
            elif row.geometry.geom_type == 'MultiPolygon':
                # If the geometry is a MultiPolygon
                for poly in row.geometry.geoms:  # Loop through each polygon
                    fig.add_trace(go.Scattermapbox(
                        lat=list(poly.exterior.coords.xy[1]),  # Convert to list
                        lon=list(poly.exterior.coords.xy[0]),  # Convert to list
                        mode='lines',  # Draw the polygon outline
                        fill='toself',  # Fill the polygon
                        fillcolor=transparent_fillcolor,  # Transparent fill color
                        line=dict(color=region_color, width=1),  # Outline color and width
                        name=row['Region1']  # Optional: Name for the legend
                    ))

    # If the download button is clicked, return the data for download
    if n_clicks > 0:
        download_data = combined_df.to_csv(index=False)
        return fig, dict(content=download_data, filename="used_data.csv")

    return fig, None

# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode="external", port = 8085)

Dash app running on http://127.0.0.1:8085/
